# Librerías

In [1]:
import re 
import pickle
import numpy as np
from glob import glob

# Funciones útiles

In [2]:
def limpiar_text(text: str)-> str:
    text = text.strip()
    text = text.replace('(','').replace(')', '')
    text = re.sub(r'\s+', ' ', text)

    text = re.sub(r'\s+[bcd]+\s+', ' ', text)
    text = re.sub(r' a ([A-Z])', r' \1', text)
    return text

In [3]:
def partir_versiculos(text: str) -> list:
    versiculos = np.array([int(i) for i in re.findall(r'\n(\d+)', text)])
    versiculos = range(1, np.max(versiculos)+1)

    versiculos_idioma = []

    for i in versiculos[:-1]:
        if i ==  versiculos[-1]:
            ver = '\n '.join(text.split('\n' + str(i))[1:])
        else:
            ver = '\n '.join(text.split('\n' + str(i))[1:])
            ver = ver.split('\n' + str(i+1))[0]
    
        ver = limpiar_text(ver)
        versiculos_idioma.append(ver)
    
    return versiculos_idioma

In [4]:
def clean_raw_text(text):
    text = re.sub(r'\n (\d+)', r'\n\1', text)
    return text 

# Limpieza de los versículos que están juntos

In [5]:
lista = [
    ('MAT', 28, 'Mateo'),
    ('MRK', 16, 'Marcos'),
    ('LUK', 24, 'Lucas'),
    ('JHN', 21, 'Juan'),
    ('ACT', 28, 'Hechos de los apostoles'),
    ('ROM', 16,'Roma'),
    ('1CO', 16, 'Corintio'),
    ('2CO', 13, 'Corintio2'),
    ('GAL', 6, 'Galacia'),
    ('EPH', 6, 'Efeso'),
    ('PHP', 4, 'Filipos'),
    ('COL', 4, 'Colosa'),
    ('1TH', 5, 'Tesalonica'),
    ('2TH', 3, '2Tesalonica'),
    ('1TI', 6, 'Timoteo'),
    ('2TI', 4, '2Timoteo'),
    ('TIT', 3, 'Tito'),
    ('PHM', 1, 'Filemon'),
    ('HEB', 13, 'Hebreokana'),
    ('JAS', 5, 'Santiago'),
    ('1PE', 5, 'Pedro'),
    ('2PE', 3, '2Pedro'),
    ('1JN', 5, '1 Juan'),
    ('2JN', 1, '2 Juan'),
    ('3JN', 1, '3 Juan'),
    ('JUD', 1, 'Judas'),
    ('REV', 22, 'Apocalipsis')
]

In [8]:
for cod, num, folder in lista:
    traduccion = []
    for i in range(1,num+1):
        path = f'../data/biblia/{folder}/'

        with open(path + f'{cod} {i} - esp.txt', 'r') as f:
            esp = ''.join(f.readlines())
        with open(path + f'{cod} {i} - wayuu.txt', 'r') as f:
            wayuu = ''.join(f.readlines())

        # Buscamos los versículos que es Wayuu están pegados de la forma 1-5
        # Esto quiere decir que deben juntar todos en un solo versículo (1)
        # Y en el correspondiente de español se deben 

        patron = r'\n(\d+)-\n(\d+)'
        coincidencias_wayuu = re.findall(patron, wayuu)

        if coincidencias_wayuu:
            for coincidencia in coincidencias_wayuu:
                print(cod, i)
                numero1, numero2 = coincidencia
                # Reemplazar la coincidencia por el primer número
                wayuu = wayuu.replace(f"{numero1}-\n{numero2}", str(numero1), 1)

                for i in range(int(numero1) + 1, int(numero2)+1):
                    esp = esp.replace(str(i), "", 1)

            with open(path + f'{cod} {i} - esp.txt', 'w') as f:
                f.write(esp)
            
            with open(path + f'{cod} {i} - wayuu.txt', 'w') as f:
                f.write(wayuu)

MAT 5
MAT 7
MAT 4
MAT 8
MAT 14
MAT 11
MAT 12
MAT 13
MAT 14
MAT 4
MAT 15
MAT 22
MAT 24
MAT 5
MAT 27
MRK 3
MRK 5
MRK 4
MRK 7
MRK 12
MRK 19
MRK 13
MRK 14
MRK 52
MRK 15
MRK 41
MRK 16
LUK 1
LUK 4
LUK 9
LUK 27
LUK 33
LUK 37
LUK 40
LUK 47
LUK 52
LUK 70
LUK 72
LUK 75
LUK 2
LUK 2
LUK 5
LUK 7
LUK 31
LUK 3
LUK 4
LUK 7
LUK 11
LUK 19
LUK 22
LUK 5
LUK 24
LUK 6
LUK 8
LUK 29
LUK 42
LUK 9
LUK 31
LUK 39
LUK 10
LUK 11
LUK 10
LUK 48
LUK 12
LUK 30
LUK 36
LUK 38
LUK 48
LUK 50
LUK 13
LUK 13
LUK 25
LUK 14
LUK 16
LUK 21
LUK 17
LUK 13
LUK 16
LUK 21
LUK 18
LUK 5
LUK 19
LUK 10
LUK 21
LUK 21
LUK 23
LUK 26
LUK 22
LUK 4
LUK 6
LUK 22
LUK 30
LUK 37
LUK 23
LUK 19
LUK 45
LUK 24
LUK 10
LUK 23
JHN 2
JHN 2
JHN 3
JHN 4
JHN 2
JHN 8
JHN 45
JHN 5
JHN 4
JHN 38
JHN 7
JHN 10
JHN 11
JHN 12
JHN 11
JHN 16
JHN 6
ACT 1
ACT 2
ACT 5
ROM 1
ROM 7
ROM 2
ROM 15
ROM 3
ROM 14
ROM 20
ROM 22
ROM 4
ROM 5
ROM 8
ROM 15
ROM 21
ROM 5
ROM 7
ROM 3
ROM 16
ROM 18
ROM 8
ROM 25
ROM 27
ROM 9
ROM 3
ROM 9
ROM 12
ROM 18
ROM 21
ROM 28
ROM 11
ROM 6
ROM 10
ROM 1

In [27]:
for cod, num, folder in lista:
    traduccion = []
    for i in range(1,num+1):
        path = f'../data/biblia/{folder}/'

        with open(path + f'{cod} {i} - esp.txt', 'r') as f:
            esp = ''.join(f.readlines())
        with open(path + f'{cod} {i} - wayuu.txt', 'r') as f:
            wayuu = ''.join(f.readlines())

        patron = r'\n\d+-\n'
        coincidencias = re.findall(patron, texto, re.MULTILINE)
        
        
        wayuu = clean_raw_text(wayuu)
        esp = clean_raw_text(esp)

        wayuu = partir_versiculos(wayuu)
        esp = partir_versiculos(esp)

        if coincidencias_es is not None or coincidencias_way is not None:
            print(f'Revisar cap {i} de {cod}. Hay coincidencias de n-m')

        elif len(wayuu) == len(esp):
            for w, e in zip(wayuu, esp): 
                traduccion.append({'wayuu': w, 'esp': e})
        else:
            print(f'Revisar cap {i} de {cod}',  len(wayuu), len(esp))

        
    
    with open(path + 'results.pickle', 'wb') as f:
        pickle.dump(traduccion, f)

Revisar cap 6 de MAT 33 41
Revisar cap 28 de MAT 19 47
Revisar cap 5 de LUK 38 45
Revisar cap 7 de LUK 499 49
Revisar cap 19 de LUK 47 54
Revisar cap 13 de JHN 37 44
Revisar cap 21 de JHN 152 74
Revisar cap 1 de ACT 119 63
Revisar cap 5 de ACT 399 41
Revisar cap 7 de ACT 399 59
Revisar cap 13 de ACT 449 59
Revisar cap 19 de ACT 40 39
Revisar cap 23 de ACT 199 34
Revisar cap 24 de ACT 26 29
Revisar cap 27 de ACT 275 43
Revisar cap 15 de 1CO 499 57
Revisar cap 13 de 2CO 13 12
Revisar cap 3 de GAL 429 28
Revisar cap 1 de JUD 199 24
Revisar cap 1 de REV 19 21
Revisar cap 11 de REV 259 18
Revisar cap 12 de REV 259 17
Revisar cap 13 de REV 665 17
Revisar cap 14 de REV 299 19
Revisar cap 21 de REV 199 26
Revisar cap 22 de REV 20 24
